**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-a4k7tpoh
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-a4k7tpoh
  Resolved https://github.com/Kaggle/learntools.git to commit ab3f19a77b8a92c7337c9fd15d84602e7320a077
  Preparing metadata (setup.py) ... - done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268981 sha256=0d39a69d792ef40d013a4b09624dbea61539e1674ce0501a2e878065756913f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-rzpaf22j/wheels/dd/d7/6b/0fc758f52767fd281d6dceded6757c6cb5bb90ccd2dbb1de9f
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,65c58f1e86d5bcd6d32bb2588db7ac8c240fb88f,f22048010592ab0cbbe57cc8e7b79daa1616239d8d8bec...,2013-05-16 02:15:00+00:00,2013-05-16 02:30:00+00:00,1020.0,0.0,NaN,NaN,6.0,28.0,...,0.0,17.25,Cash,Taxi Affiliation Services,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),41.874005,-87.663518,POINT (-87.6635175498 41.874005383)
1,9f1fd4f4349ba47f41d9761444d87882b62952e4,4c2ec0420ff3c82c7c0e76e2e6cf96208c2cff8b766c06...,2013-05-18 03:15:00+00:00,2013-05-18 03:30:00+00:00,660.0,0.0,1.703107e+10,1.703106e+10,7.0,6.0,...,1.0,10.05,Cash,Taxi Affiliation Services,41.929078,-87.646293,POINT (-87.6462934762 41.9290776551),41.936159,-87.661265,POINT (-87.6612652184 41.936159071)
2,ae1c85756f50305de0628fb8762a870247839691,cd75f70d5f3c873de240d29a244e101e2808b0a4dd2b42...,2013-05-14 10:00:00+00:00,2013-05-14 10:00:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,7.05,Credit Card,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
3,d98df904cc2766de073ed98045c275d948df5b88,aa2d1be97b83e16428fb4294532f135175cc12db45c063...,2013-05-17 21:45:00+00:00,2013-05-17 22:00:00+00:00,480.0,0.0,1.703184e+10,1.703124e+10,24.0,24.0,...,0.0,7.25,Cash,Blue Ribbon Taxi Association Inc.,41.898306,-87.653614,POINT (-87.6536139825 41.8983058696),41.906026,-87.675312,POINT (-87.6753116216 41.906025969)
4,4cb992130a4ae00766aee6bdfb2298625d004f47,b22cbb92aaf4615f4339fde442e692eda22a822ecaef28...,2013-05-18 22:45:00+00:00,2013-05-18 23:00:00+00:00,660.0,0.0,1.703184e+10,1.703184e+10,8.0,8.0,...,0.0,7.25,Cash,Blue Ribbon Taxi Association Inc.,41.904935,-87.649907,POINT (-87.6499072264 41.9049353016),41.901567,-87.638404,POINT (-87.6384040116 41.9015669095)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp > '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-03-16,98023.857143
1,2016-03-09,98175.571429
2,2016-02-21,89790.714286
3,2016-01-19,85156.571429
4,2016-01-21,83164.428571


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [5]:
# Amend the query below
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,10.0,2013-10-03 02:00:00+00:00,2013-10-03 02:00:00+00:00,1
1,10.0,2013-10-03 02:00:00+00:00,2013-10-03 02:00:00+00:00,1
2,10.0,2013-10-03 05:15:00+00:00,2013-10-03 05:30:00+00:00,3
3,10.0,2013-10-03 05:30:00+00:00,2013-10-03 05:45:00+00:00,4
4,10.0,2013-10-03 07:45:00+00:00,2013-10-03 09:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [6]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0137c4250e786bfc0cc2fc3ae88ac37cb1b5e501486a03...,2013-10-03 13:45:00+00:00,2013-10-03 14:00:00+00:00,255.0
1,044aa88179fa43c628fe5a956567224cf0b1b1ad5994f3...,2013-10-03 12:30:00+00:00,2013-10-03 12:30:00+00:00,345.0
2,044aa88179fa43c628fe5a956567224cf0b1b1ad5994f3...,2013-10-03 21:45:00+00:00,2013-10-03 21:45:00+00:00,555.0
3,146666225d80f45da0a30dc129dc4929426b5248cb6bbb...,2013-10-03 16:00:00+00:00,2013-10-03 16:15:00+00:00,240.0
4,1ed802e8138d030626cb7276ceadd721c743526af747ac...,2013-10-03 10:00:00+00:00,2013-10-03 10:00:00+00:00,510.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*